# **BDTs at work: the $\Omega$ analysis**

The goal of this tutorial is to provide an example of binary classification with machine learning techniques applied to an ALICE analysis. This tutorial is based on the measurement of the invariant mass of the $\mathrm{\Omega}$ , through its cascade decay channel $\mathrm{\Omega^-} \rightarrow \mathrm{\Lambda} + K^- \rightarrow p + \pi^- + K^-$. We will need two samples: 
- Real data: Pb--Pb collisions at $s_{\sqrt{NN}} = 5.02$ TeV (LHC18qr, subsample)
- Anchored MC production: LHC21l5

At the end of the tutorial we will be able to see the peak of the $\mathrm{\Omega}$ !


<img src="img/omega_dec.jpg" 
     align="left" 
     width="400" />

#### First, we need some libraries ###

In [ ]:
### standard sci-py libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import uproot ### to read, convert, inspect ROOT TTrees


One tip before starting: to access the documentation associated to each function we are going to call just type Shift+Tab after the first parenthesis of the function

## Reading trees with uproot, handling them with pandas

Uproot (https://github.com/scikit-hep/uproot4) is a Python package that provides tools for reading/writing ROOT files using Python and Numpy (does not depend on ROOT) and is primarly intended to stream data into machine learning libraries in Python.

In [ ]:
## inspect tree branches

In [ ]:
mc_file = uproot.open("AnalysisResults-mc_reduced.root")

In [ ]:
mc_file.keys()

In [ ]:
mc_file["XiOmegaTree"].keys()

In [ ]:
## convert the tree into a dictionary of numpy arrays

In [ ]:
numpy_mc = mc_file["XiOmegaTree"].arrays(library="np")

In [ ]:
## print the array

Not easy to handle! Better to use Pandas. Pandas is a library that provides data structures and analysis tools for Pyhton. The two primary data structures of pandas are Series (1-dimensional) and DataFrame (2-dimensional) and we will work with them.
- Series are 1-dimensional ndarray with axis labels.
- DataFrames are 2-dimensional tabular data structure with labeled axes (rows and columns).

For more details: https://pandas.pydata.org/pandas-docs/stable/

In [ ]:
## same exercise as before, but use pandas this time

In [ ]:
pd_mc = mc_file["XiOmegaTree"].arrays(library="pd")

In [ ]:
pd_mc.head()

In [ ]:
pd_mc.columns ## the suffix MC indicates the generated quantity

#### Query and eval operations to apply selections and generate new columns

In [ ]:
## let's focus only on Omegas
pd_mc.query("isOmega and abs(pdg)==3334", inplace=True)

In [ ]:
## have a look at the momentum and decay length generated distributions
plt.hist(pd_mc["ctMC"], bins=1000, range=[1,15]);
plt.yscale("log")
plt.xlabel(r"$ct$ (cm)");
plt.ylabel("Counts");

In [ ]:
## select only the MC particles that are reconstructed, but keep the full df
df_reco = pd_mc.query("isReconstructed")

In [ ]:
## compute the relative momentum resolution using eval

In [ ]:
df_reco.eval("momRes = (pt - ptMC)/ptMC", inplace=True)

In [ ]:
plt.hist(df_reco["momRes"], bins=800, range=[-0.1,0.1]);
plt.xlabel(r"$(p_{\mathrm{T}} ^{gen} - p_{\mathrm{T}} ^{rec}) / p_{\mathrm{T}} ^{gen}$");

## Machine Learning

Supervised learning is a subcategory of ML well known in HEP. Supervised learning
algorithms are employed for discriminating between two or more classes, signal and
background in our case, starting from a set of examples called training set. Each
element of the training sample, a $\Omega$ candidate in this tutorial, has a label
containing its class (signal / background), which is known a priori: the training
process fixes the internal parameters of the learning algorithm in order to maximize
the separation power among the classes. The goal of the training is to teach to the model a common pattern in data that can be used to classify properly an independent sample, in our case the real data sample. The output of the supervised model, or score, is evaluated starting from the candidate properties, which are called features. The score is related to the candidate probability of belonging to the different classes


In this tutorial Boosted Decision Trees (BDTs) will be used for tagging real $\Omega$ candidates. The core of every BDT model is the decision tree algorithm (DT). A
DT is a flowchart-like binary structure where an internal node represents a feature(or
candidate), the branch represents a decision rule, and each leaf node represents the
outcome. The topmost node in a decision tree is known as the root node. The DT
works by combining a sequence of simple binary tests (each branch of the tree), to
classify a data point in terms of its features. Each test consists in a linear threshold
applied to one of the features which helps the model to predict the belonging class
of every candidate. The training of a DT consists in the automatic procedure that builds the tree recursively starting from the training set. The main flaw of the DT is that it is prone to the so-called overfitting: this means that the model is able to perfectly classify the training set if deep enough (the depth
is defined as the length of the longest path from a root to a leaf), but it does not generalize well to new data. Overfitting occurs when the model memorizes the training
set rather than learning a general pattern in the data. To overcome this problem,
BDT algorithms combine numerous shallow trees using for each a subsample of
features. In particular, in the boosting procedure the DTs are constructed sequentially
taking care of compensating the misclassified candidates of the previous trees. The
resulting model, the BDT, maintains high performances both on the training and the
test set.




<img src="img/xgb_score.png" 
     align="left" 
     width="400" />

First we will import hipe4ml: https://github.com/hipe4ml/hipe4ml

This is a package developed in ALICE containing useful methods and classes for dealing with ML analyses. Two main classes are implemented:
- TreeHandler, wrapping uproot and pandas methods: allows for conversion and handling of the training samples
- ModelHandler, a common interface for many ML methods

In [ ]:
from hipe4ml.model_handler import ModelHandler
from hipe4ml.tree_handler import TreeHandler
from hipe4ml import analysis_utils
from hipe4ml import plot_utils

In [ ]:
### Read TTrees with hipe4ml

In [ ]:
hdl_mc = TreeHandler("AnalysisResults-mc_reduced.root", "XiOmegaTree")
hdl_data = TreeHandler("AnalysisResults_reduced.root", "XiOmegaTree")

In [ ]:
### select only reconstructed MC Omega candidates: they will be our signal sample for the training
hdl_mc.apply_preselections("abs(pdg)==3334 and isOmega==1 and isReconstructed==1") 

In [ ]:
hdl_mc.print_summary()

In [ ]:
### How to select the background? We can take it from the candidates available in
### the sidebands of our real data sample! 

In [ ]:
hdl_bkg = hdl_data.apply_preselections("mass < 1.660 or mass > 1.685", inplace=False)

In [ ]:
## now we remove the background from the data sample
hdl_data.apply_preselections("mass > 1.660 and mass < 1.685", inplace=True)

Data prepared! Now, before the training, we need to visualize the feature properties.
hipe4ml plot utilities allow for
- comparing the features of different samples (plot_distr)
- evaluate the correlations among the features

In [ ]:
cols_to_be_compared = ['pt', 'ct', 'mass', 'dcaBachPV', 'dcaV0PV', 'dcaV0piPV', 'dcaV0prPV', 'dcaV0tracks', 
                 'dcaBachV0','cosPA', 'cosPAV0', 'tpcNsigmaV0Pr']

In [ ]:
## some matplotlib tuning is needed to display all the features
plot_utils.plot_distr([hdl_mc, hdl_bkg], cols_to_be_compared, 
                      bins=50, labels=['Signal', 'Background'],
                      log=True, density=True, figsize=(12, 12), alpha=0.3, grid=False);

#### Some questions....
- Which variables do we expect to be relevant for the training? Can we use all of them? Pros  and cons?
- why do we see some spikes in the DCA?

And correlations are important as well: the model can potentially exploit them to perform a better classification. Moreover, there could be some potentially dangerous correlations as those with the invariant mass of the particle of interest

In [ ]:
plot_utils.plot_corr([hdl_mc, hdl_bkg], cols_to_be_compared, labels=['Signal', 'Background']);

Considerations? Doubts? Let's now define the training columns and build the training sample

In [ ]:
training_cols = ['dcaBachPV', 'dcaV0PV', 'dcaV0piPV', 'dcaV0prPV', 'dcaV0tracks', 
                 'dcaBachV0','cosPA', 'cosPAV0', 'tpcNsigmaV0Pr']

Now we split our data in a training and test set. To do it, we use the train_test_generator function from hipe4ml

In [ ]:
train_test_data = analysis_utils.train_test_generator([hdl_bkg, hdl_mc], [0, 1], test_size=0.5, random_state=42)

In [ ]:
### let's print the train_test_data variable and see what we have

## Training and testing a BDT

We will use the BDT of XGBoost (https://github.com/dmlc/xgboost): boosting is implemented with a gradient descent method. It features few hyperparameters that can be tuned to improve the performance and reduce the overfitting, even if the algorithm works smoothly out of the box.



In [ ]:
import xgboost as xgb

In [ ]:
xgb_model = xgb.XGBClassifier()

In [ ]:
### wrap the classifier into the ModelHandler

In [ ]:
model_hdl = ModelHandler(xgb_model, training_cols)

In [ ]:
model_hdl.fit(train_test_data[0], train_test_data[1])

Training Done! Let's apply the model to the test sample

In [ ]:
score_test = model_hdl.predict(train_test_data[2])

In [ ]:
#### plot the score distribution
plt.hist(score_test, bins=100, label='full sample', alpha=0.5, density=True);

Two peaks clearly distinguishable: will they be corresponding to the signal and the background? Let's plot the two distribution separately

In [ ]:
plt.hist(score_test[train_test_data[3]==0], bins=100, label='background', alpha=0.5, density=True);
plt.hist(score_test[train_test_data[3]==1], bins=100, label='signal', alpha=0.5, density=True);
plt.legend();


Well separated peaks on the test set! This is what we want.
Now we can evaluate the performance on the test set: we can use the ROC curve, which is built by plotting
the true positive rate (TPR) against the false negative rate (FPR) as a function of the
score threshold, where TPR and FPR are defined as:


$TPR=\frac{\sum TP}{\sum TP + \sum FN} \hspace{2cm} FPR=\frac{\sum FP}{\sum FP + \sum TN} $

and in our case represent the signal selection and the background rejection
efficiencies respectively as a function of the score. The most common way employed to evaluate the performance of a BDT is to
compute the area under the ROC curve, called AUC: a perfect classifier will have
a ROC AUC equal to 1, whereas a random classifier will have a ROC AUC equal
to 0.5.



In [ ]:
plot_utils.plot_roc(train_test_data[3], score_test);

Repeat this exercise with the training set: what do you get?

In [ ]:
score_train = model_hdl.predict(train_test_data[0])

In [ ]:
plot_utils.plot_roc(train_test_data[1], score_train);

Training ROC-AUC is slightly higher than the test set one. This is a systematic behaviour due to the small presence of overfitting. We can see it also by plotting the BDT output for the training and test set distributions

In [ ]:
plot_utils.plot_output_train_test(model_hdl, train_test_data, density=True, bins=100, logscale=True);

Now, before applying the BDT to data we can have a look at which variables are relevant for the training. We will use the feature importance implemented in the SHAP library (https://github.com/slundberg/shap). In the context of machine learning, the Shapley value is used to evaluate the contribution of each feature to the model output, and it is calculated by averaging the marginal contributions of each feature to the model output. The marginal contribution of a feature is the difference in the model output when the feature is present or absent. The variables that are
more important for the model are those that have a higher marginal contribution, and Shapley values consequently.

In hipe4ml the function plot_feat_imp implements the algorithm: try to use it!

In [ ]:
plot_utils.plot_feature_imp(train_test_data[2], train_test_data[3], model_hdl) 

Two plots given: how to interpret them?

**Bonus** : repeat the training using a different model and compare its performance with the XGB BDT. All the sklearn and keras (NN) models can be used to feed the ModelHandler

### Applying the BDT

Now that the model is tested we can use it to classify the real data sample. Try to evaluate the invariant mass as a function of the score distribution

In [ ]:
hdl_data.apply_model_handler(model_hdl)

In [ ]:
plt.hist(hdl_data.apply_preselections("model_output>-3", inplace=False)["mass"], bins=100);

In [ ]:
plt.hist(hdl_data.apply_preselections("model_output>0.5", inplace=False)["mass"], bins=100);

plt.hist(hdl_data.apply_preselections("model_output>4", inplace=False)["mass"], bins=100);

In [ ]:
### very nice! But how to decide which threshold is the best one? Many methods can be used, but it is important to evaluate the selection efficiency
### as a function of the score. Excercise: write a function for computing the BDT efficiency vs Score. Would you compute it on the training or the test sets?

In [ ]:
### hipe4ml has a function to do that! 
eff_array, score_array = analysis_utils.bdt_efficiency_array(train_test_data[3], score_test, 1000)

In [ ]:
plot_utils.plot_bdt_eff(score_array, eff_array);

### **Bonus**: optimize your BDT

The XGBoost Classifier has many hyperparameters that control the complexity of
the model. Few of them are listed here, for a complete description see :

- n_estimators: Number of trees in the BDT
- max_depth: Maximum depth of a tree
- eta: learning rate of the algorithm. It controls the step size of the gradient descent algorithm


The optimisation of the hyperparameters is a key step to obtain the best performance from the algorithm and prevent overfitting. In hipe4ml the Optuna package is employed for the optimisation through the method ModelHandler.optimize_params_optuna. (https://github.com/optuna/optuna)

The Optuna package provides a wide choice of algorithms for the hyperparameter optimization. The default one is the TPESampler, which is known to provide robust performance in few iterations. The difference between other approaches, like grid search or random search, and the TPESampler optimisation is that the latter takes into account past evaluations when choosing the hyperparameter set to evaluate next.

A set of hyperparameters should be tested on different samples to avoid overfitting problems. Since the number of events is limited, an approach called cross validation is used. It has been proved that the cross validation removes the dependence of the model on the data sample.

In the cross validation procedure, the original sample is divided in n parts called folds (in this case 5 folds are used). For each set of hyperparameters, n-1 folds are used for the optimisation and the remaining one as test. This operation is repeated after permuting the folds used for optimisation and for testing and the final result is the mean value of all the permutations.

The ModelHandler automatically updates the hyperparameters after their optimisation.

In [ ]:
### Excercise: try to optimize your hyperparams with optuna and compare the performance! Be aware: the optimisation is CPU expensive and take some time